In [ ]:
# Create a _netrc file
#!echo "machine urs.earthdata.nasa.gov login YOURUSERNAME password YOURPASSWORD" > ~/_netrc
#!chmod 600 ~/_netrc

In [3]:
import numpy as np
import pandas as pd
import requests
import shutil
import time
import xarray as xr

from concurrent.futures import ThreadPoolExecutor
from getpass import getpass
from http.cookiejar import CookieJar
from io import StringIO
from itertools import repeat
from pathlib import Path
from platform import system
from netrc import netrc
from os.path import expanduser, basename, isfile, isdir, join
from tqdm import tqdm
from urllib import request

_netrc = join(expanduser('~'), "_netrc" if system()=="Windows" else ".netrc")

# ----- Helper Subroutines -----

# Function to log into NASA EarthData
def setup_earthdata_login_auth(url: str='urs.earthdata.nasa.gov'):
    # look for the netrc file and use the login/password
    try:
        username, _, password = netrc(file=_netrc).authenticators(url)

    # if file is not found, prompt user for the login/password
    except (FileNotFoundError, TypeError):
        print('Please provide Earthdata Login credentials for access.')
        username, password = input('Username: '), getpass('Password: ')

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, url, username, password)
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar()
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener)

# Functions to make API calls
def set_params(params: dict):
    params.update({'scroll': "true", 'page_size': 2000})
    return {par: val for par, val in params.items() if val is not None}

def get_results(params: dict, headers: dict=None):
    response = requests.get(url="https://cmr.earthdata.nasa.gov/search/granules.csv",
                            params=set_params(params),
                            headers=headers)
    return response, response.headers

def get_granules(params: dict):
    response, headers = get_results(params=params)
    scroll = headers['CMR-Scroll-Id']
    hits = int(headers['CMR-Hits'])
    if hits==0:
        raise Exception("No granules matched your input parameters.")
    df = pd.read_csv(StringIO(response.text))
    while hits > df.index.size:
        response, _ = get_results(params=params, headers={'CMR-Scroll-Id': scroll})
        data = pd.read_csv(StringIO(response.text))
        df = pd.concat([df, data])
    return df

# Function to download single files
def download_file(url: str, output_dir: str, force: bool=False):
    """
    url (str): the HTTPS url from which the file will download
    output_dir (str): the local path into which the file will download
    force (bool): download even if the file exists locally already
    """
    if not isdir(output_dir):
        raise Exception(f"Output directory doesn't exist! ({output_dir})")

    target_file = join(output_dir, basename(url))

    # if the file has already been downloaded, skip
    if isfile(target_file) and force is False:
        print(f'\n{basename(url)} already exists, and force=False, not re-downloading')
        return 0

    with requests.get(url) as r:
        if not r.status_code // 100 == 2:
            raise Exception(r.text)
            return 0
        else:
            with open(target_file, 'wb') as f:
                total_size_in_bytes= int(r.headers.get('content-length', 0))
                for chunk in r.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)

                return total_size_in_bytes

# Function to download all URLs concurrently
def download_files_concurrently(dls, download_dir, force=False, max_workers=6):
    start_time = time.time()

    # Use multiple threads for concurrent downloads
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # progress bar
        results = list(tqdm(executor.map(download_file, dls, repeat(download_dir), repeat(force)), total=len(dls)))

        # add up the total downloaded file sizes
        total_download_size_in_bytes = np.sum(np.array(results))
        # calculate total time spent in the download
        total_time = time.time() - start_time

        print('\n=====================================')
        print(f'total downloaded: {np.round(total_download_size_in_bytes/1e6,2)} Mb')
        print(f'avg download speed: {np.round(total_download_size_in_bytes/1e6/total_time,2)} Mb/s')

# ----- Main Script -----

# OISSS dataset shortname
ShortName = "OISSS_L4_multimission_monthly_v2"

# Specify desired date range - adjust as needed
# This dataset starts from August 2011
StartDate = "2016-01-01"
EndDate = "2016-12-31"

# Define download directory
download_root_dir = Path('./Earth_Data')  # Change this for directory
download_dir = download_root_dir / ShortName

# Create the download directory
download_dir.mkdir(exist_ok=True, parents=True)
print(f'Download directory: {download_dir}')

# Log into Earthdata
setup_earthdata_login_auth()

# Search parameters for the dataset
input_search_params = {
    'ShortName': ShortName,
    'temporal': ",".join([StartDate, EndDate])
}

print(f"Searching for {ShortName} data between {StartDate} and {EndDate}")

# Query CMR for the OISSS Dataset
try:
    grans = get_granules(input_search_params)
    num_grans = len(grans['Granule UR'])
    print(f'Found {num_grans} matching granules')

    # Convert the URLs to a list
    dls = grans['Online Access URLs'].tolist()

    if num_grans > 0:
        print(f'First file: {dls[0]}')

        # Download the granules with concurrent downloads
        max_workers = 6  # Adjust based on your internet connection
        force = True    # Set to True to force redownload of existing files

        print("Starting download...")
        download_files_concurrently(dls, download_dir, force, max_workers)

        # List downloaded files
        salinity_files = list(download_dir.glob('*nc'))
        print(f'Total downloaded files: {len(salinity_files)}')

        # Optionally, open and plot data
        if len(salinity_files) > 0:
            print("You can now open the files for analysis with:")
            print("xds = xr.open_mfdataset(salinity_files, parallel=True)")
            print("xds.SSS.mean('time').plot(figsize=[20,10])")
    else:
        print("No files found for the specified date range.")

except Exception as e:
    print(f"Error: {e}")

Download directory: Earth_Data\OISSS_L4_multimission_monthly_v2
Searching for OISSS_L4_multimission_monthly_v2 data between 2016-01-01 and 2016-12-31
Found 9 matching granules
First file: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OISSS_L4_multimission_monthly_v2/OISSS_L4_multimission_global_monthly_v2.0_2016-01.nc
Starting download...


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:06<00:00,  1.39it/s]


total downloaded: 61.97 Mb
avg download speed: 9.57 Mb/s
Total downloaded files: 20
You can now open the files for analysis with:
xds = xr.open_mfdataset(salinity_files, parallel=True)
xds.SSS.mean('time').plot(figsize=[20,10])


In [4]:
import os
from pathlib import Path

# This will print the absolute path
print(os.path.abspath("Earth_Data/OISSS_L4_multimission_monthly_v2"))

# List the files to confirm they're there
files = list(Path("Earth_Data/OISSS_L4_multimission_monthly_v2").glob("*.nc"))
for file in files:
    print(file)

C:\Users\jspier\pre\Earth_Data\OISSS_L4_multimission_monthly_v2
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-01.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-02.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-03.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-04.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-05.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-06.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-08.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-09.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimission_global_monthly_v2.0_2015-10.nc
Earth_Data\OISSS_L4_multimission_monthly_v2\OISSS_L4_multimissi

In [6]:
import numpy as np
import pandas as pd
import requests
import shutil # Not used in the provided snippet, but kept if originally intended
import time
import xarray as xr # Not used in the download script part, but kept from original imports

from concurrent.futures import ThreadPoolExecutor
from getpass import getpass
from http.cookiejar import CookieJar
from io import StringIO
from itertools import repeat
from pathlib import Path
from platform import system
from netrc import netrc
from os.path import expanduser, basename, isfile, isdir, join # basename, isfile, isdir, join are used
from tqdm import tqdm
from urllib import request # Used for auth setup

# Determine the netrc file path based on OS
_netrc_file = join(expanduser('~'), "_netrc" if system()=="Windows" else ".netrc")

# ----- Helper Subroutines -----

# Function to log into NASA EarthData
def setup_earthdata_login_auth(url: str='urs.earthdata.nasa.gov', netrc_path: str=_netrc_file):
    """Sets up authentication for NASA Earthdata using netrc or user input."""
    try:
        username, _, password = netrc(file=netrc_path).authenticators(url)
        if username is None or password is None: # Handles cases where netrc exists but no entry for the url
            raise FileNotFoundError # Treat as if netrc didn't have the info
        print(f"Using credentials from netrc file: {netrc_path}")
    except (FileNotFoundError, TypeError, netrc.NetrcParseError):
        print(f"Could not find valid credentials in netrc file ({netrc_path}) or file not found.")
        print('Please provide Earthdata Login credentials for access.')
        username, password = input('Username: '), getpass('Password: ')

    manager = request.HTTPPasswordMgrWithDefaultRealm()
    manager.add_password(None, url, username, password) # The first None means it's a global password for the URL
    auth = request.HTTPBasicAuthHandler(manager)
    jar = CookieJar() # To handle cookies
    processor = request.HTTPCookieProcessor(jar)
    opener = request.build_opener(auth, processor)
    request.install_opener(opener) # Install this opener for all 'request' calls

# Functions to make API calls to CMR
def set_params(params: dict):
    """Updates search parameters with defaults for CMR query."""
    # Defaults for CMR search, including enabling scrolling and setting a large page size
    default_params = {'scroll': "true", 'page_size': 2000}
    # Update the provided params with these defaults; provided params will override if keys conflict
    # before this line, but here we ensure our defaults are set.
    # A better way is to start with defaults and update with params.
    final_params = default_params.copy()
    final_params.update(params)
    # Filter out any parameters that are None
    return {par: val for par, val in final_params.items() if val is not None}

def get_results(params: dict, headers: dict=None):
    """Submits a GET request to CMR and returns the response and headers."""
    cmr_url = "https://cmr.earthdata.nasa.gov/search/granules.csv"
    try:
        # Pass processed parameters to the GET request
        response = requests.get(url=cmr_url, params=set_params(params), headers=headers)
        response.raise_for_status() # Raise an exception for HTTP errors (4xx or 5xx)
        return response, response.headers
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error during CMR Query: {e}")
        print(f"Response content: {e.response.text}")
        raise
    except requests.exceptions.RequestException as e:
        print(f"Request Exception during CMR Query: {e}")
        raise


def get_granules(params: dict):
    """
    Retrieves granule information from CMR based on search parameters.
    Handles scrolling if results span multiple pages.
    """
    print(f"Initial CMR search parameters being used: {params}")
    response, headers = get_results(params=params) # Initial request

    # Extract scroll ID and total hits from response headers
    scroll_id = headers.get('CMR-Scroll-Id')
    hits = int(headers.get('CMR-Hits', 0))

    print(f"CMR reported {hits} total matching granules.")

    if hits == 0:
        raise Exception("No granules matched your input parameters according to CMR.")

    # Read the CSV data from the first response
    try:
        df = pd.read_csv(StringIO(response.text))
    except pd.errors.EmptyDataError:
        print("CMR returned an empty CSV response for the first page, but reported hits > 0. This is unusual.")
        df = pd.DataFrame() # Start with an empty dataframe

    print(f"Found {len(df)} granules on the first page.")

    # Scroll for more results if needed
    # Note: For page_size=2000 and only 12 expected granules, this loop likely won't run.
    while hits > len(df.index) and scroll_id:
        print(f"Continuing scroll. Current granules: {len(df)}, Total hits: {hits}, Scroll ID: {scroll_id}")
        # For subsequent scroll requests, CMR expects only the scroll_id in headers.
        # Params for get_results should be empty or minimal as the search context is in the scroll_id.
        # The `set_params({})` will still add `scroll=true` and `page_size`,
        # but CMR should prioritize the scroll_id.
        response, headers_scroll = get_results(params={}, headers={'CMR-Scroll-Id': scroll_id})
        
        # Update scroll_id from the new headers for the next potential iteration
        # Some CMR versions might not return a new scroll_id on every scroll page,
        # or might expect the same scroll_id to be used. This assumes a new one or reuse of old.
        # If headers_scroll.get('CMR-Scroll-Id') is None, it might mean the end of scroll session.
        current_scroll_id = headers_scroll.get('CMR-Scroll-Id')
        if not current_scroll_id or current_scroll_id != scroll_id: # If scroll_id changes or disappears
            scroll_id = current_scroll_id # Update it
            if not scroll_id:
                print("Scroll ID disappeared, stopping scroll.")
                break

        try:
            data = pd.read_csv(StringIO(response.text))
            if not data.empty:
                df = pd.concat([df, data], ignore_index=True)
                print(f"Fetched page via scroll, total granules in DataFrame now: {len(df)}")
            else:
                print("Scroll request returned no new data, stopping scroll.")
                break # No new data
        except pd.errors.EmptyDataError:
            print("Scroll request returned an empty CSV, stopping scroll.")
            break # Empty response

    if len(df.index) != hits:
        print(f"Warning: Final granule count in DataFrame ({len(df.index)}) does not match CMR-Hits ({hits}).")
        
    return df


# Function to download single files
def download_file(url: str, output_dir_path: Path, force: bool=False):
    """
    Downloads a single file from a URL to an output directory.

    url (str): The HTTPS URL from which the file will download.
    output_dir_path (Path): The local Path object for the directory.
    force (bool): Download even if the file exists locally already.
    """
    if not output_dir_path.is_dir():
        # This should have been created by download_dir.mkdir() earlier
        print(f"Output directory {output_dir_path} doesn't exist! Attempting to create.")
        try:
            output_dir_path.mkdir(parents=True, exist_ok=True)
        except Exception as e:
            raise Exception(f"Failed to create output directory {output_dir_path}: {e}")

    target_file = output_dir_path / basename(url) # Use Path object for joining

    if target_file.is_file() and not force:
        print(f'\n{target_file.name} already exists, and force=False, not re-downloading.')
        return 0 # Return 0 bytes downloaded

    # Use the session created by request.install_opener() implicitly with requests.get
    # The setup_earthdata_login_auth configures the global urllib.request opener,
    # requests library needs its own auth handling if not using the default opener
    # or if cookies are essential and managed by urllib.
    # For simplicity and to leverage setup_earthdata_login_auth directly with `requests`,
    # it's better if setup_earthdata_login_auth directly configures `requests.Session`
    # or if download_file uses `urllib.request.urlopen`.
    # However, the original code uses requests.get() here, assuming cookies set by
    # install_opener() might be picked up or that basic auth is sufficient.
    # Let's try to use the installed opener more directly for robustness:
    
    try:
        # We use requests.get as per original code, assuming the session/cookie handling
        # from urllib.request setup is somehow bridged or not strictly needed for these direct HTTPS downloads
        # once authenticated for the session. If complex cookie handling is needed, this might need
        # to use urllib.request.urlopen(url) or a shared requests.Session.
        # For now, we keep `requests.get(url, stream=True)` for chunked download,
        # assuming authentication is handled for the domain.
        with requests.get(url, stream=True, timeout=30) as r: # Added stream=True and timeout
            r.raise_for_status() # Raise an exception for HTTP errors
            
            with open(target_file, 'wb') as f:
                total_size_in_bytes = int(r.headers.get('content-length', 0))
                # Use shutil.copyfileobj for potentially more efficient streaming
                # shutil.copyfileobj(r.raw, f) # Alternative if r.iter_content is problematic
                for chunk in r.iter_content(chunk_size=8192): # Increased chunk size
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
            
            # Verify file size if content-length was provided
            if total_size_in_bytes != 0 and target_file.stat().st_size != total_size_in_bytes:
                print(f"Warning: Downloaded file {target_file.name} size {target_file.stat().st_size} "
                      f"does not match expected size {total_size_in_bytes}.")
            elif total_size_in_bytes == 0 and target_file.stat().st_size == 0:
                 print(f"Warning: Downloaded file {target_file.name} is empty (0 bytes). URL: {url}")


            return target_file.stat().st_size # Return actual downloaded size

    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP Error downloading {url}: {e}")
        print(f"Response content: {e.response.text if e.response else 'No response content'}")
        if target_file.is_file(): # Remove partially downloaded file
            target_file.unlink(missing_ok=True)
        return 0 # Indicate failure or 0 bytes
    except requests.exceptions.RequestException as e:
        print(f"\nError downloading {url}: {e}")
        if target_file.is_file(): # Remove partially downloaded file
            target_file.unlink(missing_ok=True)
        return 0 # Indicate failure or 0 bytes
    except Exception as e:
        print(f"\nAn unexpected error occurred downloading {url}: {e}")
        if target_file.is_file(): # Remove partially downloaded file
            target_file.unlink(missing_ok=True)
        return 0


# Function to download all URLs concurrently
def download_files_concurrently(dls, download_dir_path: Path, force=False, max_workers=6):
    """Downloads files concurrently using a thread pool."""
    start_time = time.time()
    results = []

    print(f"\nStarting concurrent download of {len(dls)} files to {download_dir_path} with {max_workers} workers...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Prepare arguments for executor.map
        # The download_file function will be called with (url, download_dir_path, force)
        # repeat(download_dir_path) and repeat(force) create iterators that yield the same value
        # for each call.
        # tqdm wraps the executor.map to provide a progress bar.
        results_iterator = executor.map(download_file, dls, repeat(download_dir_path), repeat(force))
        
        # tqdm processes the iterator and shows progress
        # We need to convert the iterator to a list to ensure all tasks are completed
        # and exceptions are raised if they occurred in threads.
        try:
            results = list(tqdm(results_iterator, total=len(dls), desc="Downloading files"))
        except Exception as e:
            print(f"An error occurred during concurrent downloads: {e}")
            # Potentially some downloads succeeded, some failed. Results might be partial.
            # The 'results' list will contain exceptions for tasks that failed if they weren't caught inside download_file.

    total_download_size_in_bytes = np.sum(np.array(results, dtype=np.int64)) # Ensure int64 for sum
    total_time = time.time() - start_time
    num_successful_downloads = sum(1 for r in results if isinstance(r, (int, float)) and r > 0)


    print('\n=====================================')
    print(f'Download process complete.')
    print(f'Successfully downloaded {num_successful_downloads}/{len(dls)} files.')
    print(f'Total downloaded data: {np.round(total_download_size_in_bytes / 1e6, 2)} MB')
    if total_time > 0:
      print(f'Average download speed: {np.round(total_download_size_in_bytes / 1e6 / total_time, 2)} MB/s (for successful downloads)')
    else:
      print(f'Total time was negligible.')
    print('=====================================')
    return [d for i, d in enumerate(dls) if isinstance(results[i], (int, float)) and results[i] > 0]


# ----- Main Script -----
def main():
    # OISSS dataset shortname
    ShortName = "OISSS_L4_multimission_monthly_v2"

    # Specify desired date range - adjust as needed
    # This dataset is cited to start from August 2011
    StartDate = "2016-01-01"
    EndDate = "2016-12-31" # Inclusive end date for the search

    # Define download directory using Pathlib
    download_root_dir = Path('./Earth_Data')  # Change this for your desired root directory
    download_dir = download_root_dir / ShortName

    # Create the download directory if it doesn't exist
    download_dir.mkdir(exist_ok=True, parents=True)
    print(f'Download directory: {download_dir.resolve()}')

    # Log into Earthdata - this sets up global authentication for urllib.request
    # `requests` might need explicit auth or session handling if cookies are critical
    # and not bridged from urllib's global state.
    setup_earthdata_login_auth()

    # Search parameters for the dataset
    input_search_params = {
        'ShortName': ShortName,
        'temporal': f"{StartDate},{EndDate}" # CMR expects temporal as a comma-separated string
    }

    print(f"\nSearching CMR for {ShortName} data between {StartDate} and {EndDate}...")

    try:
        grans_df = get_granules(input_search_params)
        
        if grans_df.empty or 'Online Access URLs' not in grans_df.columns:
            print("No granules found or 'Online Access URLs' column missing in CMR response.")
            return

        # Filter out rows where 'Online Access URLs' might be NaN or empty strings if any
        grans_df.dropna(subset=['Online Access URLs'], inplace=True)
        dls = grans_df['Online Access URLs'].tolist()
        num_grans_to_download = len(dls)

        print(f'\nCMR search yielded {num_grans_to_download} granules with downloadable URLs.')

        if num_grans_to_download == 0:
            print("No files to download.")
            return

        print("\nURLs to be downloaded:")
        for i, url in enumerate(dls):
            print(f"{i+1}: {url}")
        
        # Expected number of months (for a full year)
        start_month = pd.to_datetime(StartDate).month
        end_month = pd.to_datetime(EndDate).month
        start_year = pd.to_datetime(StartDate).year
        end_year = pd.to_datetime(EndDate).year
        
        expected_months = 0
        if start_year == end_year:
            expected_months = (end_month - start_month + 1)
        else: # Multi-year, more complex, but for 2016-01-01 to 2016-12-31 it's 12
            expected_months = (12 - start_month + 1) + (end_year - start_year - 1) * 12 + end_month
        
        print(f"Expected number of monthly granules for the period: {expected_months}")
        if num_grans_to_download < expected_months:
            print(f"Warning: Found fewer granules ({num_grans_to_download}) than expected ({expected_months}). Some data may be missing from the source for this period.")


        # Download the granules with concurrent downloads
        max_workers = 4  # Adjust based on your internet connection and server limits (too many can cause issues)
        force_redownload = False # Set to True to force redownload of existing files

        print(f"\nProceeding to download {num_grans_to_download} files. Force redownload: {force_redownload}")
        successfully_downloaded_urls = download_files_concurrently(dls, download_dir, force_redownload, max_workers)

        # List downloaded files (based on successful downloads reported by the function)
        # This is more accurate than globbing if some downloads failed.
        print(f'\nSuccessfully downloaded {len(successfully_downloaded_urls)} files:')
        # For verification, list files in the directory as well
        if download_dir.exists():
            print(f"\nFiles found in download directory ({download_dir}):")
            downloaded_files_in_dir = sorted(list(download_dir.glob('*.nc')))
            for f_path in downloaded_files_in_dir:
                print(f_path.name)
            print(f"Total .nc files in directory: {len(downloaded_files_in_dir)}")
        
            if len(downloaded_files_in_dir) < expected_months and len(downloaded_files_in_dir) == len(successfully_downloaded_urls):
                 print("\n--- IMPORTANT ---")
                 print("If the number of downloaded files is less than expected, "
                       "it's highly likely that the data for the missing months was not found by the CMR search.")
                 print("This means the data might not be available from NASA for your specified product and date range.")
                 print("You can verify this by manually searching on the Earthdata Search portal:")
                 print("https://search.earthdata.nasa.gov/search")
                 print(f"using ShortName: {ShortName} and dates: {StartDate} to {EndDate}")
                 print("---")


        # Optionally, open and plot data (example)
        if len(successfully_downloaded_urls) > 0:
            print("\nExample of how you can open the files for analysis (if they are NetCDF):")
            # Construct file paths for successfully downloaded NetCDF files
            salinity_nc_files = [download_dir / basename(url) for url in successfully_downloaded_urls if url.endswith('.nc')]
            if salinity_nc_files:
                # Ensure files actually exist before trying to open
                salinity_nc_files_exist = [f for f in salinity_nc_files if f.is_file()]
                if salinity_nc_files_exist:
                    print("xds = xr.open_mfdataset(salinity_files, engine='netcdf4', parallel=True, combine='by_coords')")
                    print("# Example plot, actual variable name might differ, check your .nc file structure")
                    print("# xds['your_salinity_variable_name'].mean('time').plot(figsize=[12,6])")
                else:
                    print("No successfully downloaded .nc files found to demonstrate opening.")
            else:
                print("No .nc files were in the list of successfully downloaded URLs.")


    except Exception as e:
        print(f"\nAn error occurred in the main script: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    main()

Download directory: C:\Users\jspier\pre\Earth_Data\OISSS_L4_multimission_monthly_v2
Using credentials from netrc file: C:\Users\jspier\_netrc

Searching CMR for OISSS_L4_multimission_monthly_v2 data between 2016-01-01 and 2016-12-31...
Initial CMR search parameters being used: {'ShortName': 'OISSS_L4_multimission_monthly_v2', 'temporal': '2016-01-01,2016-12-31'}
CMR reported 9 total matching granules.
Found 9 granules on the first page.

CMR search yielded 9 granules with downloadable URLs.

URLs to be downloaded:
1: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OISSS_L4_multimission_monthly_v2/OISSS_L4_multimission_global_monthly_v2.0_2016-01.nc
2: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OISSS_L4_multimission_monthly_v2/OISSS_L4_multimission_global_monthly_v2.0_2016-03.nc
3: https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OISSS_L4_multimission_monthly_v2/OISSS_L4_multimission_global_monthly_v2.0_2016-05.nc



Download process complete.
Successfully downloaded 0/9 files.
Total downloaded data: 0.0 MB
Average download speed: 0.0 MB/s (for successful downloads)

Successfully downloaded 0 files:

Files found in download directory (Earth_Data\OISSS_L4_multimission_monthly_v2):
OISSS_L4_multimission_global_monthly_v2.0_2015-01.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-02.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-03.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-04.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-05.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-06.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-08.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-09.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-10.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-11.nc
OISSS_L4_multimission_global_monthly_v2.0_2015-12.nc
OISSS_L4_multimission_global_monthly_v2.0_2016-01.nc
OISSS_L4_multimission_global_monthly_v2.0_2016-03.nc
OISSS_L4_multimission_global_monthly_v2.0_